In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [0]:
movies_languages_schema = StructType([
    StructField("movieId", IntegerType(), True),
    StructField("languageId", StringType(), True),
    StructField("languageRoleId", StringType(), True)
    ])
movies_languages_df = spark.read\
    .option("multiLine", True)\
    .json(f"{bronze_folder_path}/{v_file_date}/movie_language/", schema=movies_languages_schema)

In [0]:
from pyspark.sql.functions import lit, current_timestamp
movies_languages_renamed_df = movies_languages_df\
    .withColumnsRenamed({"movieId": "movie_id", "languageId": "language_id"})\
    .withColumns({"environment": lit(v_environment), "ingestion_date": current_timestamp(), "file_date": lit(v_file_date)})

In [0]:
movies_languages_renamed_df.drop("languageRoleId").display()

In [0]:
#overwrite_partition(movies_languages_renamed_df, "movie_silver", "movie_language")

In [0]:
merge_condicion = "tgt.language_id = src.language_id AND tgt.movie_id = src.movie_id AND tgt.file_date = src.file_date"
merge_delta_lake(movies_languages_renamed_df, "movie_silver", "movie_language", silver_folder_path, merge_condicion, "file_date")

In [0]:
%sql
SELECT file_date, COUNT(1)
FROM movie_silver.movie_language
GROUP BY file_date;

In [0]:
%sql
SELECT * FROM movie_silver.movie_language